## Modelamiento de tópicos

El modelado de tópicos es un tipo de modelado estadístico para descubrir los temas abstractos que ocurren en una colección de documentos.  Todos estos modelos se basan en el mismo supuesto básico:

* Cada documento consiste en una mezcla de temas, y
* Cada tema consiste en una colección de palabras.

En otras palabras, el modelamiento de tópicos o temas se basa en la idea de que la semántica de nuestro documento en realidad está gobernada por algunas variables ocultas o latentes que no estamos observando. Como resultado, el objetivo principal de estos modelos es descubrir estas variables latentes que dan forma al significado de nuestro documento y nuestro corpus. 

In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib
import matplotlib.pyplot as plt
from pprint import pprint

import re

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.test.utils import datapath


import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline 

with open('data1.pickle', 'rb') as f:
    [ids1, date1, texts1, ortexts1, times1, favtimes1, follower_count1, location1, lang1, i] = pickle.load(f)
print(len(texts1))
#Query: 
#       referendumperu OR referendum2018 OR referendum OR bicameralidad OR (reeleccion de congresistas) OR 
#       (reeleccion inmediata) OR (financiamiento de campaña) OR (reforma de justicia) OR 
#       (reforma constitucional)

#Tweets desde Sun Dec 09 06:33:26 +0000 2018 hasta Sun Dec 09 15:54:04 +0000 2018

with open('data2.pickle', 'rb') as f:
    [ids2, date2, texts2, ortexts2, times2, favtimes2, follower_count2, location2, lang2, i] = pickle.load(f)
print(len(texts2))
#Query: 
#       referendumperu OR referendum2018 OR referendum OR bicameralidad OR (reeleccion de congresistas) OR 
#       (reeleccion inmediata) OR (financiamiento de campaña) OR (reforma de justicia) OR 
#       (reforma constitucional)

#Tweets desde Sun Dec 09 15:54:50 +0000 2018 hasta Mon Dec 10 02:08:20 +0000 2018

In [ ]:
def ext_prop(x1,x2):
    temp = []
    temp.extend(x1)
    temp.extend(x2)
    return temp

In [ ]:
#Juntando la data

ids = ext_prop(ids1,ids2)
dates = ext_prop(date1,date2)
texts = ext_prop(texts1,texts2)
ortexts = ext_prop(ortexts1,ortexts2)
times = ext_prop(times1,times2)
favtimes = ext_prop(favtimes1,favtimes2)
followers = ext_prop(follower_count1,follower_count2)
locations = ext_prop(location1,location2)
langs = ext_prop(lang1,lang2)

In [ ]:
data = {'id':ids, 'fecha': dates,'texto':texts,'ortext':ortexts,'retweets':times,'seguidores':followers,'localiz':locations}
df = pd.DataFrame(data)

In [ ]:
#Quitando los retweets
df1 = df[df['ortext'].str.startswith('RT')==False]
print('Cantidad de tweets que no son RT:', len(df1))

El procesamiento del lenguaje natural (NLP) es un campo de investigación que presenta muchos desafíos, como la comprensión del lenguaje natural. Los stopwords se pueden filtrar del texto a procesar. No hay una lista universal de stopwords en la investigación de NLP.  sin embargo, el módulo nltk contiene una lista de stopwords.



Cuando instalamos  paquetes NLTK usando `nltk.download()`. Uno de los paquetes fue WordNet. WordNet es una base de datos creada para el procesamiento del lenguaje natural. Incluye grupos de sinónimos y una breve definición.

La derivación regresiva significa eliminar afijos de las palabras y encontrar la palabra raíz. Los motores de búsqueda utilizan esta técnica al indexar páginas, puesto que muchas personas escriben versiones diferentes para la misma palabra y todas derivan de la palabra raíz. Lemantizar palabras es similar a la derivación regresiva, y el proceso de lematizar con WordNet se usa `NLTK WordNet`, 

In [ ]:
import string
import nltk 
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer

# Limpieza de datos
stop = set(stopwords.words('spanish'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

def clean(documento):
    stop_free = " ".join([i for i in documento.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

In [ ]:
documento_clean_1 = [clean(documento).split() for documento in texts1]  


Creamos un diccionario y el corpus necesarios para el topics modeling.

In [ ]:
id2word = corpora.Dictionary(documento_clean_1)
texts = documento_clean_1
corpus = [id2word.doc2bow(documento) for documento in documento_clean_1]

`Gensim` crea una identificación única para cada palabra en el documento. El corpus producido es un mapeo de (`word_id`, `word_frequency`).

Esto se utiliza como entrada por el modelo LDA.

In [ ]:
# Forma legible del corpus

[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

## Técnicas del modelado de tópicos

### LSA

Imagina por un momento que tienes una montaña de libros y artículos ante ti, y quieres extraer la esencia de todas esas palabras sin tener que pasar meses leyéndolas. Aquí es donde entra en juego el Análisis Semántico Latente (LSA, por sus siglas en inglés), una técnica poderosa en el campo del procesamiento de lenguaje natural que ayuda a las máquinas a 'comprender' y organizar grandes volúmenes de texto. 

LSA es una técnica de procesamiento de lenguaje natural que descubre la estructura semántica oculta en grandes colecciones de texto. Funciona al construir una matriz término-documento, donde filas representan términos únicos y columnas representan documentos, rellenando esta matriz con la frecuencia de cada término en cada documento. Sin embargo, el verdadero poder de LSA emerge con la aplicación de SVD. 

Recuerda que SVD es una herramienta matemática que descompone la matriz término-documento en tres matrices más simples $U$, $\Sigma$ y $V^T$, cada una capturando aspectos distintos de los datos originales. $U$ y $V^T$ contienen los vectores singulares izquierdos y derechos, respectivamente, representando los términos y documentos en un espacio de características reducido. La matriz Σ, diagonal, contiene los valores singulares, que son indicativos de la importancia (o "peso") de cada dimensión en capturar la estructura de los datos. 

Esta transformación no solo reduce la dimensionalidad de los datos, facilitando el manejo computacional y la visualización, sino que también minimiza el ruido, destacando las relaciones semánticas más significativas.  

Aplicar SVD en LSA implica esencialmente filtrar el ruido y resaltar la información más relevante. Esto nos permite trabajar en un espacio donde los conceptos similares se mapean cercanamente, facilitando la detección de relaciones semánticas incluso cuando los documentos no comparten palabras clave. 

La representación de la matriz término-documento mediante SVD transforma el caótico mar de datos en un conjunto ordenado de vectores, donde cada vector representa un concepto o tema. Esta representación compacta y esencial permite realizar cálculos de similitud y agrupación con una eficiencia impresionante, abriendo nuevas posibilidades para la exploración y el análisis de textos. 


### PLSA

Al explorar la vasta extensión del procesamiento del lenguaje natural, nos encontramos con una evolución significativa del Análisis Semántico Latente: el Análisis Semántico Latente Probabilístico (PLSA). Imagina entrar en una cocina donde cada ingrediente tiene una probabilidad de ser elegido para una receta; de manera similar, PLSA introduce un modelo probabilístico para entender cómo los temas contribuyen a la generación de documentos. 

Los fundamentos de PLSA se basan en la idea de que un documento se puede modelar como una mezcla de varios temas o tópicos, donde cada tema es una distribución sobre palabras. Este enfoque introduce una capa adicional de complejidad y precisión al análisis de textos, permitiendo que las máquinas no solo vean qué palabras aparecen juntas, sino también estimen cómo estos agrupamientos de palabras se asocian con temas particulares. 

El corazón del concepto matemático de PLSA es la **maximización de la verosimilitud de los datos observados** (las palabras en los documentos) bajo el modelo. Esto se logra a través de un **algoritmo de maximización de expectativas (EM)**, que itera entre estimar las distribuciones de temas en los documentos (paso E) y ajustar los parámetros del modelo para maximizar la probabilidad de los datos observados dado estos temas (paso M). 

La diferencia clave entre LSA y PLSA radica en esta naturaleza probabilística. Mientras que LSA busca espacios semánticos latentes a través de métodos algebraicos, PLSA se adentra en la probabilidad de que ciertos temas estén presentes en un documento, ofreciendo un modelo más flexible y detallado para la estructuración de la información. 


### Asignación de Dirichlet Latente (LDA) 

Este modelo se basa en la premisa de que los documentos son colecciones de temas, donde un tema se define como una distribución de probabilidades sobre un conjunto de palabras. Esta perspectiva permite a LDA descubrir los temas subyacentes en una colección de textos de manera automática, revelando las proporciones en que cada tema contribuye a los documentos. 

LDA se basa en la **inferencia Bayesiana** para estimar las distribuciones latentes que mejor expliquen las observaciones (las palabras en los documentos). Esto permite modelar la variabilidad en la proporción de temas por documento y la variabilidad de palabras por tema de manera que captura la incertidumbre inherente y la variabilidad natural en los datos textuales. 

La gran intuición detrás de LDA es considerar cada documento como una mezcla de varios temas, y cada palabra en el documento como una muestra de uno de estos temas. Esta aproximación modela la generación de documentos de una manera que captura la complejidad y variabilidad del lenguaje real, permitiendo una representación más rica y flexible del contenido textual.  

Aunque LDA comparte similitudes con LSA y PLSA en su objetivo de identificar estructuras temáticas latentes, se distingue por su enfoque en la modelización de temas como distribuciones explícitas sobre palabras y documentos. Este enfoque probabilístico proporciona una base más sólida para el manejo de la ambigüedad y la polisemia en el lenguaje, ofreciendo un modelo más coherente y versátil para el análisis de textos. 


### Inferencia en LDA 

Sumergiéndonos en las profundidades de la Asignación de Dirichlet Latente (LDA), nos enfrentamos al desafío crucial de la inferencia: el proceso mediante el cual desciframos la composición temática de documentos basándonos en la distribución de palabras. Imagina ser un arqueólogo tratando de reconstruir historias a partir de fragmentos de cerámica; de manera similar, la inferencia en LDA busca reconstruir los temas subyacentes de los documentos a partir de las "huellas" dejadas por las palabras. 

El corazón de este proceso es un algoritmo que itera a través de cada documento, asignando cada palabra a un tema basándose en la probabilidad de que la palabra pertenezca a ese tema, y a su vez, actualizando la probabilidad de los temas en todo el documento. Este método iterativo, aunque computacionalmente intensivo, permite a LDA ajustar y refinar sus estimaciones de la distribución de temas dentro de los documentos y de palabras dentro de los temas. 

La belleza de LDA reside en su capacidad para calcular la probabilidad conjunta de una colección de temas que generan una colección de documentos, considerando tanto la probabilidad de que un tema produzca ciertas palabras como la probabilidad de que un documento contenga ciertos temas. Este delicado equilibrio probabilístico es lo que permite a LDA ofrecer una visión coherente y detallada de la estructura temática de los textos. 

Una vez entrenado el modelo LDA con un conjunto de datos, podemos aplicarlo a documentos nuevos, inferir su composición temática basándonos en las distribuciones aprendidas. Esto es como descifrar el código genético de un texto para entender su linaje temático, proporcionando conocimiento valioso para la clasificación de documentos, la recomendación de contenidos y el análisis de tendencias. 

### Hierarchical Latent Dirichlet Allocation (HLDA) 

 
El concepto detrás de HLDA se basa en una extensión del modelo Latent Dirichlet Allocation (LDA), que identifica temas en conjuntos de documentos. Sin embargo, HLDA lleva esto un paso más allá al introducir una estructura jerárquica en la identificación de temas. 

Los fundamentos matemáticos de HLDA se basan en un modelo de Dirichlet de procesos, donde cada documento puede ser visto como una mezcla de varios temas, y cada tema es a su vez una mezcla de palabras. Sin embargo, a diferencia de LDA, en HLDA estos temas están organizados en una jerarquía, permitiendo que temas más generales se desglosen en temas más específicos, de manera similar a cómo se ramifican las ramas de un árbol. Matemáticamente, HLDA utiliza el **Proceso de Dirichlet** para asignar distribuciones de probabilidad a temas a diferentes niveles de la jerarquía, y luego emplea el muestreo de Gibbs, una técnica de Monte Carlo basada en cadenas de Markov, para inferir la estructura jerárquica de temas a partir de los datos. Este proceso iterativo ajusta la asignación de temas a documentos y la asignación de palabras a temas, optimizando la jerarquía temática para reflejar mejor las relaciones inherentes en el conjunto de datos. 


### Dynamic Topic Model (DTM) 

¿Alguna vez te has preguntado cómo las conversaciones y los temas de interés han evolucionado a lo largo del tiempo? En un mundo donde el cambio es la única constante, entender la dinámica de estos cambios en los discursos y documentos se convierte en un reto fascinante. Aquí es donde el Dynamic Topic Model (DTM) entra en juego, actuando como una máquina del tiempo para el análisis de textos. 

El Dynamic Topic Model (DTM) es una extensión sofisticada del modelo Latent Dirichlet Allocation (LDA) diseñada específicamente para capturar cómo los temas evolucionan a lo largo del tiempo. Mientras que LDA puede identificar temas dentro de un conjunto de documentos, DTM añade una dimensión temporal, permitiéndonos ver cómo estos temas cambian, emergen o desaparecen a través de diferentes períodos. 

La base matemática de DTM se centra en incorporar el tiempo como una variable crucial en el modelo de asignación de temas.  Hace uso de procesos estocásticos para modelar la evolución temporal de los temas. En concreto, DTM emplea Cadenas de Markov para representar cómo las distribuciones de palabras asociadas a cada tema pueden cambiar de un tiempo t a t+1 Cada tema en el modelo es asociado con una distribución de probabilidad sobre un vocabulario fijo, y esta distribución se ajusta en cada periodo temporal, permitiendo que el modelo capture tendencias emergentes, temas en declive y la fluctuación en la importancia de las palabras dentro de un tema. 

Además, DTM utiliza el método de **inferencia de variación Bayesiana**, un enfoque computacionalmente eficiente para estimar las distribuciones de temas a lo largo del tiempo. Este método permite calcular aproximaciones de las distribuciones posteriores de los parámetros del modelo, dados los datos observados, facilitando la identificación de la estructura temática dinámica del corpus. 

Las aplicaciones prácticas de DTM son inmensamente valiosas y diversas. Desde el seguimiento de la evolución de discursos políticos, pasando por la detección de tendencias emergentes en publicaciones científicas, hasta el análisis de la popularidad de temas en medios de comunicación social, DTM ofrece conocimiento único sobre cómo y por qué ciertos temas capturan la atención pública en momentos específicos. 
. 

### Correlated Topic Model (CTM) 

En el entramado de ideas que forman nuestra comunicación escrita, no todos los temas existen en aislamiento. ¿Te has dado cuenta de cómo ciertos temas tienden a aparecer juntos, como si estuvieran danzando al mismo ritmo? Este fenómeno, donde los temas no solo coexisten, sino que también se influyen mutuamente, captura la esencia del Correlated Topic Model (CTM). CTM lleva el análisis de textos más allá de la simple identificación de tópicos, adentrándose en el complejo mundo de sus interrelaciones. 

A diferencia de su predecesor, el modelo Latent Dirichlet Allocation (LDA), que considera la aparición de temas en documentos de manera independiente, el CTM introduce la capacidad de modelar la correlación entre temas. Esto se logra mediante el uso de la distribución logística normal para modelar las proporciones de temas en los documentos, permitiendo que el modelo capture cómo la presencia de un tema puede influir en la presencia de otro. 

La innovación matemática del CTM radica en su enfoque para manejar la correlación. Al permitir que los temas estén correlacionados, el modelo puede reflejar con mayor precisión la realidad de cómo se generan los documentos. Por ejemplo, en un conjunto de artículos sobre tecnología, es probable que los temas "inteligencia artificial" y "robótica" aparezcan correlacionados, reflejando su interconexión en el mundo real. 

El proceso matemático del CTM se basa en la **inferencia variacional**, un método eficiente para aproximar las distribuciones posteriores de interés. Este enfoque permite estimar las correlaciones entre temas y cómo estos se distribuyen a lo largo de los documentos del corpus. La inferencia variacional en CTM se ocupa de optimizar una función objetivo que mide la discrepancia entre la distribución posterior real y su aproximación, asegurando que el modelo capture de manera efectiva las complejidades de las correlaciones temáticas. 

Las aplicaciones prácticas del CTM son extensas y potentes. Desde mejorar sistemas de recomendación que pueden sugerir contenidos más relevantes basados en la comprensión de correlaciones temáticas, hasta enriquecer análisis de sentimientos donde la presencia de ciertos temas afecta el tono y la interpretación del texto. El CTM es particularmente útil en campos como el análisis de medios, la investigación de mercados y el estudio de tendencias culturales y sociales, donde comprender las relaciones entre temas puede ofrecer conocimiento profundo y accionables.

### Parallel Latent Dirichlet Allocation (PLDA) 


El PLDA extiende el modelo Latent Dirichlet Allocation (LDA) tradicional al introducir una estructura que permite su ejecución en paralelo. En el corazón del PLDA yace una división inteligente del proceso de asignación de tópicos, donde el conjunto de datos se divide en segmentos más pequeños que pueden ser procesados simultáneamente en múltiples núcleos de procesador o máquinas. Esta paralelización no solo acelera significativamente el análisis, sino que también hace escalable el modelado de temas para enfrentar el crecimiento exponencial de los datos. 

La innovación de PLDA se refleja en su estructura computacional, que se apoya en la descomposición de la tarea de modelado de temas en subprocesos que pueden ejecutarse concurrentemente. Esta aproximación reduce drásticamente los tiempos de procesamiento, permitiendo que análisis que anteriormente tomaban días, ahora se puedan completar en horas o incluso minutos, dependiendo de la infraestructura disponible. 

Las aplicaciones prácticas de PLDA son vastas, especialmente en entornos donde el volumen de datos y la necesidad de resultados oportunos son críticos. Desde el análisis en tiempo real de publicaciones en redes sociales hasta el procesamiento de documentos académicos y la exploración de grandes archivos digitales, PLDA ofrece a investigadores y analistas la capacidad de desentrañar la estructura temática de sus datos a una velocidad sin precedentes.  

#### Pachinko Allocation Model (PAM) 

¿Has jugado alguna vez al pachinko, ese juego japonés donde las pequeñas bolas metálicas caen a través de un laberinto de clavijas, siguiendo trayectorias influenciadas por la casualidad y la física? El Pachinko Allocation Model (PAM) toma su nombre de este juego, y al igual que las inesperadas rutas de las bolas en el pachinko, PAM nos permite explorar las complejas y multifacéticas relaciones entre temas en grandes colecciones de documentos. 

PAM es una evolución tanto del Latent Dirichlet Allocation (LDA) como del Hierarchical Latent Dirichlet Allocation (HLDA), diseñado para capturar no solo jerarquías de temas sino también dependencias arbitrarias entre ellos. Mientras que modelos anteriores podrían ser restringidos a jerarquías fijas o suponer independencia entre temas, PAM introduce una flexibilidad sin precedentes, permitiendo que cualquier tema pueda influir en la aparición de otros a través de una red de relaciones. 

El marco matemático de PAM se basa en la analogía con los diagramas acíclicos dirigidos (DAGs), donde los nodos representan temas y las aristas las relaciones entre ellos. Al permitir múltiples capas de temas, con temas específicos influenciados por varios temas más generales, PAM puede modelar la generación de documentos con una riqueza y una precisión extraordinarias. Este modelo es capaz de reflejar cómo un documento sobre tecnología puede abarcar temas tan variados como la inteligencia artificial, la ética y la economía, todos interconectados en un complejo tejido temático. 

Las aplicaciones de PAM son tan variadas como los patrones que puede descubrir. Desde el análisis detallado de tendencias en datos de redes sociales, donde los intereses de las personas se entrelazan de maneras complejas, hasta la mejora de sistemas de recomendación que pueden discernir con mayor precisión las preferencias de los usuarios basándose en patrones temáticos intrincados. PAM también brilla en el ámbito académico y de investigación, donde puede ayudar a desentrañar las relaciones entre disciplinas y subdisciplinas, ofreciendo una visión más holística y conectada del conocimiento. 


### DAGs y su rol en modelos de tópicos

Los DAGs son fundamentales en modelos avanzados de asignación de temas, como el Pachinko Allocation Model (PAM), donde facilitan la representación de relaciones complejas y jerárquicas entre temas. En esencia, un DAG es un grafo que consta de nodos (representando temas, en este contexto) y aristas dirigidas (indicando relaciones de influencia o pertenencia) que no forman ciclos, asegurando que cada camino tenga un principio y un fin claros. 

El papel de los DAGs en el modelado de tópicos es doble. Primero, permiten una representación visual y matemática de las relaciones entre temas, facilitando la comprensión de cómo los temas generales se desglosan en subtemas más específicos o cómo diferentes temas se influyen mutuamente. Segundo, los DAGs ayudan a estructurar el proceso de inferencia en el modelo, guiando el cálculo de probabilidades de que ciertos temas aparezcan en documentos, basándose en la estructura del grafo. 

Las aplicaciones prácticas de los DAGs en modelos de temas son amplias y profundas. En el ámbito académico, pueden ayudar a mapear el cuerpo de conocimiento de una disciplina, revelando subdisciplinas y cómo se relacionan entre sí. En el análisis de redes sociales, pueden ilustrar cómo se propagan las ideas a través de comunidades. Y en la industria, pueden optimizar sistemas de recomendación al entender mejor las relaciones entre preferencias de usuarios. 
 

## Modelo LDA

Supongamos que tiene el siguiente conjunto de oraciones:

* Me gusta comer choclos y plátanos.
* Comí un surtido de papaya y piña para el desayuno.
* Las chinchillas y los gatitos son lindos.
* Mi hermana adoptó un gatito ayer.
+ Mira este linda ardilla comiendo un pedazo de choclo.

LDA, es una forma de descubrir automáticamente los tópicos que contienen estas oraciones. Por ejemplo, dadas estas oraciones y solicitando 2 temas, LDA podría producir algo como

* Oraciones 1 y 2: 100% Tema A
* Oraciones 3 y 4: 100% Tema B
* Oración 5: 60% Tema A, 40% Tema B
 
* Tópico A: 30% de choclos, 15% de plátanos, 10% de desayuno, 10% de comida,… (en ese momento, se podría interpretar que el Tópico A trata sobre alimentos)
* Tópico B: 20% de chinchillas, 20% de gatitos, 20% de lindos, 15% de ardilla… (en ese momento, puedes interpretar que el Tópico B trata de animales lindos)

LDA representa los documentos como una mixtura de tópicos que entregan palabras con ciertas probabilidades. Se supone que los documentos se producen de la siguiente manera: al escribir cada documento,

* Decidimos el número de palabras `N` que tendrá el documento (por ejemplo, según una distribución de Poisson).
* Escogemos una mezcla de temas para el documento (de acuerdo con una distribución de Dirichlet sobre un conjunto fijo de temas de K). Por ejemplo, suponiendo que tengamos los dos temas de alimentos y animales lindos dados anteriormente, se puede elegir que el documento conste de `1/3` de alimentos y `2/3` de animales lindos.
* Generamos cada palabra $w_i$ en el documento de la siguiente forma:
     - Primero, escogemos un tema.
     - Uso del tópico para generar la palabra en sí. Por ejemplo, si seleccionamos el tema de alimentos, podríamos generar la palabra `choclos` con    un 30% de probabilidad `plátanos` con un 15% de probabilidad, y así sucesivamente.

Asumiendo este modelo generativo para una colección de documentos, LDA retrocede en los documentos para encontrar un conjunto de temas que probablemente hayan generado la colección.


In [ ]:
# Construyendo el modelo LDA

ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=10,  random_state=100,
                                           update_every=1,chunksize=100,passes=50, alpha='auto',
                                           per_word_topics=True)

In [ ]:
pprint(ldamodel.print_topics())
documento_lda = ldamodel[corpus]

El tópico 3 se representa como ` '0.061*"referéndum" + 0.032*"reformas" + 0.023*"va" + 0.021*"vamos" + ' '0.020*"ser" + 0.017*"¿qué" + 0.016*"4vecesno" + 0.016*"mejor" + ' '0.015*"hacemos" + 0.014*"mamarracho"'`

Significa que las 10 palabras clave principales que contribuyen a este tópico son: `referédum`, `reformas`, `va`, `4vecesno`, etc. y el peso de `coche` en el tópico 3 es `0.061`. Los pesos reflejan la importancia de una palabra clave para ese tema.


### Evaluación del modelado tópico

[Model perplexity](http://qpleple.com/perplexity-to-evaluate-topic-models/) y [topic coherence](http://qpleple.com/topic-coherence-to-evaluate-topic-models/) proporcionan una medida conveniente para juzgar qué tan bueno es el modelo de tópicos dado. 


In [ ]:
# Calculamos  Perplexity
print('\nPerplejidad: ', ldamodel.log_perplexity(corpus)) 

# Coherence score
from gensim.models import CoherenceModel
coherence_model_lda = CoherenceModel(model=ldamodel, texts=documento_clean_1, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nPuntuación de la coherencia: ', coherence_lda)

### Visualizando tópicos

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, id2word)
vis

Cada burbuja en el gráfico del lado izquierdo representa un tema. Cuanto más grande es la burbuja, más frecuente es ese tema. Usualmente un buen  modelo tendrá burbujas bastante grandes y no superpuestas dispersas en todo el gráfico en lugar de agruparse en un cuadrante. 

Un modelo con demasiados temas, generalmente tendrá muchas superposiciones, burbujas de tamaño pequeño agrupadas en una región del gráfico. Si se mueve el cursor sobre una de las burbujas, las palabras y las barras del lado derecho se actualizarán. Estas palabras son las palabras clave más importantes que forman el tema seleccionado.


### Mejorando el modelo LDA

In [ ]:
from gensim.models import CoherenceModel, LdaMulticore
from pprint import pprint

# Creación del modelo LDA usando LdaMulticore
ldamodel = LdaMulticore(corpus=corpus, num_topics=10, id2word=id2word, passes=10, workers=4)
pprint(ldamodel.show_topics(formatted=False))

In [ ]:
# Calculamos Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=documento_clean_1, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\n Puntuacion de coherencia : ', coherence_lda)

### Calculando el número óptimo de tópicos para LDA

La idea para encontrar el número óptimo de tópicos es construir muchos modelos LDA con diferentes valores de número de tópicos y elegir el que ofrezca el mayor valor `coherence`.


In [ ]:
def calcula_coherence_topicos(dictionary, corpus, texts, limit, start=2, step=3):
    valores_coherence = []
    lista_modelo = []
    for num_topicos in range(start, limit, step):
        modelo = LdaMulticore(corpus=corpus, num_topics=num_topicos, id2word=id2word, passes=10, workers=4)
        lista_modelo.append(modelo)
        coherencemodel = CoherenceModel(model=modelo, texts=texts, dictionary=dictionary, coherence='c_v')
        valores_coherence.append(coherencemodel.get_coherence())

    return lista_modelo, valores_coherence

In [ ]:
lista_modelo, valores_coherence = calcula_coherence_topicos(dictionary=id2word, corpus=corpus, 
                                                            texts=documento_clean_1, start=2, limit=40, step=6)

In [ ]:
# Mostramos un gráfico
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, valores_coherence)
plt.xlabel("Num Tópicos")
plt.ylabel("Coherence score")
plt.legend(("Valores_coherence"), loc='best')
plt.show()


In [ ]:
# Imprimimos los coherence score
for m, cv in zip(x, valores_coherence):
    print("Num Topics =", m, " tiene valor de coherence", round(cv, 4))

In [ ]:
# Seleccionamos el modelo y imprimimos los topicos 
modelo_optimo = lista_modelo[0]
modelo_topicos = modelo_optimo.show_topics(formatted=False)
pprint(modelo_optimo.print_topics(num_words=10))

Una de las aplicaciones prácticas del modelado de tópicos es determinar de qué tópico trata un documento dado. Para encontrar eso, encontramos el número de tópico que tiene el mayor porcentaje de contribución en ese documento.

In [ ]:
def topicos_oraciones(ldamodel=ldamodel, corpus=corpus, texts=documento_clean_1):
    # Salida inicial
    topicos_enviados_df = pd.DataFrame()

    # Obtener el tema principal en cada documento
    for i, fila in enumerate(ldamodel[corpus]):
        fila= sorted(fila, key=lambda x: (x[1]), reverse=True)
        # Calculamos el tópico dominante, la contribución porcentual y las palabras clave para cada documento
        for j, (num_topico, prop_topico) in enumerate(fila):
            if j == 0:  
                wp = ldamodel.show_topic(num_topico)
                palabras_topicos = ", ".join([word for word, prop in wp])
                topicos_enviados_df = topicos_enviados_df.append(pd.Series([int(num_topico), round(prop_topico,4), palabras_topicos]), 
                                                                 ignore_index=True)
            else:
                break
    topicos_enviados_df.columns = ['Topico_dominante', 'Por_Contribucion', 'Palabras_topico']

    # Agregar texto original al final de la salida.
    contenido = pd.Series(texts)
    topicos_enviados_df = pd.concat([topicos_enviados_df, contenido], axis=1)
    return(topicos_enviados_df)


df_palabras_enviados_topicos = topicos_oraciones(ldamodel=modelo_optimo, corpus=corpus, texts=documento_clean_1)

# Formato
df_topico_dominante = df_palabras_enviados_topicos.reset_index()
df_topico_dominante.columns = ['No_documento', 'Topico_dominante', 'Topicos_Por_Contribucion', 'Palabras clave', 'Texto']

# Show
df_topico_dominante.head(10)

A veces, solo las palabras clave de los tópicos pueden no ser suficientes para dar sentido a lo que trata el tópico. Por lo tanto, para ayudar a comprender el tópico, podemos  encontrar los documentos en los que más ha contribuido un tópico determinado e inferir el tópico leyendo ese documento.


**Ejercicio:** ¿Que sucede si reemplazas todas las instancias de `ldamodel` con `modelo_optimo` dentro de la función.

In [ ]:
## Tu respuesta

In [ ]:
# Agrupamos las 5 oraciones principales de cada topico
topicos_enviados_ordenados_= pd.DataFrame()

topicos_enviados_outdf_grpd = df_palabras_enviados_topicos.groupby('Topico_dominante')

for i, grp in topicos_enviados_outdf_grpd:
    topicos_enviados_ordenados_ = pd.concat([topicos_enviados_ordenados_, 
                                             grp.sort_values(['Por_Contribucion'], ascending=[0]).head(1)], 
                                            axis=0)

topicos_enviados_ordenados_.reset_index(drop=True, inplace=True)

# Formato
topicos_enviados_ordenados_.columns = ['Num_topicos', "Topicos_Por_Contribucion", "Palabras claves", "Texto"]

topicos_enviados_ordenados_.head()

Queremos entender el volumen y la distribución de los tópicos  para juzgar qué tan ampliamente se discutió. La siguiente tabla expone esa información.

In [ ]:
# Numero de documentos para cada tópico
conteo_topicos= df_palabras_enviados_topicos['Topico_dominante'].value_counts()

# Percentaje de documentos para cada tópico
contribucion_topicos = round(conteo_topicos/conteo_topicos.sum(), 4)

# Numero de topicos y palabras claves
num_palabras_topicos = df_palabras_enviados_topicos[['Topico_dominante', 'Palabras_topico']]

df_topicos_dominante = pd.concat([num_palabras_topicos, conteo_topicos, contribucion_topicos], axis=1)

df_topicos_dominante.columns = ['Topico_dominante', 'Palabras_topico', 'Num_Documentos', 'Por_Documentos']

# Crear un DataFrame final con la información agregada
df_topicos_dominante = pd.DataFrame({
    'Num_Documentos': conteo_topicos,
    'Por_Documentos': contribucion_topicos
}).reset_index()

df_topicos_dominante = pd.merge(df_topicos_dominante, num_palabras_topicos, how='left', left_on='index', right_on='Topico_dominante')
df_topicos_dominante.drop(['index'], axis=1, inplace=True)

# Mostrar los resultados
print(df_topicos_dominante)

### Ejercicio

Python ofrece varias bibliotecas para facilitar la programación en paralelo, como `multiprocessing`. Sin embargo, para tareas específicas de modelado de tópicos como LDA, algunas bibliotecas de procesamiento de lenguaje natural y aprendizaje automático, como Gensim, ya optimizan el uso de múltiples núcleos sin necesidad de implementación manual.

Completa el código siguiente para realizar LDA en paralelo con el modelo LdaMulticore, que está diseñado para aprovechar múltiples núcleos de CPU y acelerar el entrenamiento:

In [ ]:
from gensim.models import LdaMulticore
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel
import multiprocessing

# Supongamos que ya tienes tus documentos procesados en 'textos_procesados'
# textos_procesados = [['palabra1', 'palabra2', ...], [...], ...]

# Crea un diccionario Gensim a partir de los textos procesados
diccionario = Dictionary(textos_procesados)

# Filtra palabras extremadamente raras/muy comunes
diccionario.filter_extremes(no_below=10, no_above=0.5)

# Convierte los documentos a una representación vectorial
corpus = [diccionario.doc2bow(texto) for texto in textos_procesados]

# Número de tópicos
num_topics = 10

# Crea una instancia de LdaMulticore
# 'workers' es menos uno que el número de núcleos de tu CPU. Si tienes 4 núcleos, ponlo en 3.
# Esto es porque uno se deja para tareas de coordinación. Sin embargo, ajusta según tus necesidades y observaciones.
lda_model = LdaMulticore(
    corpus=corpus,
    id2word=diccionario,
    num_topics=num_topics,
    workers=min(3, multiprocessing.cpu_count() - 1),
    passes=10,
    random_state=100
)

# Evalúa el modelo con Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=textos_procesados, dictionary=diccionario, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nPuntuacion de coherencia ', coherence_lda)

# Explora los tópicos
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nPalabras: {}'.format(idx, topic))
